In [85]:
import pandas as pd
from pandas.plotting import radviz
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
#import the model we are using
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm, preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn import linear_model
from sklearn.ensemble import BaggingClassifier, GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

In [86]:
def read(file):
    return pd.read_csv(file)

#features.set_index(['Index'], inplace = True)
#features.drop(columns = ['Index'], inplace = True)

features = read('Pmax_forward_Final.csv')
features.head(5)

,α [°],H [mm],b [mm],Ls [mm],γ [N/mm3],Pmax [N],Vroot [mm3],φ [°]
0,15,90.0,19.4114,17.555563,0.000016,9.080644,4534.350557,39.33
1,15,90.0,12.9410,41.703709,0.000016,8.758713,3535.876732,39.33
2,15,90.0,19.4114,17.555563,0.000016,11.753734,8267.564868,39.33
3,15,90.0,12.9410,41.703709,0.000016,10.797683,5796.469487,39.33
4,15,90.0,19.4114,17.555563,0.000016,14.095079,23281.011643,39.33


In [87]:
#summary statistics
features.describe()
features.columns.values

array(['α [°]', 'H [mm]', 'b [mm]', 'Ls [mm]', 'γ [N/mm3]', 'Pmax [N]',
       'Vroot [mm3]', 'φ [°]'], dtype=object)

In [88]:
#pd.options.display.max_rows = 181
pd.options.display.max_columns = 30
#features.sort_values(by = "L [mm]", ascending = True, inplace = True)
#display(features)

In [89]:
#one hot encoding
pd.options.display.max_columns = 181
pd.options.display.max_rows = 181

if 'n [count]' in features.columns and 'Material' in features.columns:
    features = pd.get_dummies(features, columns = ['n [count]', 'Material'])
elif 'n [count]' in features.columns:
    features = pd.get_dummies(features, columns = ['n [count]'])
elif 'Material' in features.columns:
    features = pd.get_dummies(features, columns = ['Material'])
#features.head(5)
#display(features)
#print_shape(features)

In [90]:
#target pmax/f
pmax = np.array(features['Pmax [N]']).reshape(1, -1)

#Remove labels from the features
features = features.drop('Pmax [N]', axis = 1)
feature_list = list(features.columns)
features = StandardScaler().fit_transform(features)
pmax = Normalizer().fit_transform(pmax).reshape(-1, 1)
#pca = PCA(n_components = 11)
#principalComponents = pca.fit_transform(features)
#features = pd.DataFrame(data = principalComponents, columns = ['pc1', 'pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10','pc11'])
#display(features)
#print(pca.explained_variance_ratio_.cumsum())

#Covert to numpy arrays
features = np.array(features)


In [91]:
#split the data into training set and testing set 
train_features, test_features, train_labels, test_labels = train_test_split(features, pmax, test_size = 0.25, random_state = 42)

In [92]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (132, 7)
Training Labels Shape: (132, 1)
Testing Features Shape: (45, 7)
Testing Labels Shape: (45, 1)


In [93]:
gb = GradientBoostingRegressor(random_state=42, n_estimators=300, learning_rate=0.1, max_depth=3)
gb.fit(train_features, train_labels.ravel())

GradientBoostingRegressor(n_estimators=300, random_state=42)

In [94]:
gb_predictions = gb.predict(test_features)
print(gb_predictions)

[0.04060493 0.0276241  0.02365663 0.12310513 0.02227459 0.06421245
 0.02024699 0.13691325 0.01993967 0.03290926 0.0391802  0.04115537
 0.04460083 0.01381685 0.03521725 0.03814628 0.02957733 0.0578005
 0.02603851 0.07205508 0.13281593 0.04611679 0.07356587 0.09869766
 0.126334   0.02958733 0.12601576 0.04308508 0.18124487 0.04781114
 0.05891328 0.02838185 0.0431642  0.01726873 0.12540003 0.04594043
 0.09339729 0.01639241 0.02223675 0.06403328 0.13495542 0.03059518
 0.08263386 0.03348798 0.06156695]


In [95]:
gb_mean_absolute_error = mean_absolute_error(test_labels, gb_predictions)
print('Mean Absolute Error:', round(gb_mean_absolute_error, 5))

Mean Absolute Error: 0.00607


In [96]:
mean_square_error = mean_squared_error(test_labels, gb_predictions, squared=False)

print("Mean Square Error: ", round(mean_square_error, 5))

Mean Square Error:  0.01009


In [97]:
gb_explained_variance_score = explained_variance_score(test_labels, gb_predictions)

print("Explained variance score: ", round(gb_explained_variance_score, 7))

Explained variance score:  0.9354103
